In [2]:
import pandas as pd
import numpy as np
import torch 

In [3]:
data=pd.read_csv('skill_builder_data_corrected_collapsed.csv')

C:\Users\mpathirana\AppData\Local\Temp\ipykernel_14756\3495006265.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('skill_builder_data_corrected_collapsed.csv')


In [4]:
data

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,3,4,124564,30060,NaN,65,0,0.0,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346855,38214014,291495,96299,57830,108976,0,1,1,38234,tutor,...,0,2,38234,55692,200260.0,NaN,0,NaN,54,NaN
346856,38214016,291495,96299,57843,109015,0,1,1,6500,tutor,...,0,0,6500,55693,200299.0,NaN,0,NaN,55,NaN
346857,38214195,291503,96299,34577,54060,0,1,1,18344,tutor,...,0,3,18344,30677,NaN,0.8,0,NaN,56,NaN
346858,38214196,291503,96299,34577,54061,0,1,1,12922,tutor,...,0,2,12922,30677,NaN,-6,0,NaN,57,NaN


In [22]:
def preprocess_data(data):
    """
    Pre-process data and pad to the maximum length.
    """
    features = ['skill_id', 'correct']
    seqs = data.groupby(['user_id']).apply(lambda x: x[features].values.tolist())
    # ensure sequence is not too long
    length = min(max(seqs.str.len()), block_size)
    seqs = seqs.apply(lambda s: s[:length] + (length - min(len(s), length)) * [[-1000] * len(features)])
    return seqs

def construct_batches(raw_data, epoch = 0, val = False):
    """
    Construct batches based on tabular KT data with user_id, skill_id, and
    correctness. Pads to the minimum of the maximum sequence length and the
    block size of the transformer.
    """
    np.random.seed(epoch)
    user_ids = raw_data['user_id'].unique()
    # Loop until one epoch of training.
    for _ in range(len(user_ids) // batch_size):
        user_idx = raw_data['user_id'].sample(batch_size).unique() if not val else user_ids[_ * (batch_size // 2): (_ + 1) * (batch_size // 2)]
        filtered_data = raw_data[raw_data['user_id'].isin(user_idx)].sort_values(['user_id', 'order_id'])
        batch_preprocessed = preprocess_data(filtered_data)
        batch = np.array(batch_preprocessed.to_list())
        # Next token prediction.
        X = torch.tensor(batch[:, :-1, ..., :], requires_grad=True)
        y = torch.tensor(batch[:, 1:, ..., [0, 1]], requires_grad=True)
        for i in range(X.shape[1] // block_size + 1):
            if X[:, i * block_size: (i + 1) * block_size].shape[1] > 0:
                yield [X[:, i * block_size: (i + 1) * block_size], y[:, i * block_size: (i + 1) * block_size]]

In [23]:
batch_size=10
block_size=10
next(construct_batches(data))

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [11]:
preprocessData=preprocess_data(data)

In [20]:
preprocessData[14]

[['2_37_70', 0],
 ['2_37_70', 1],
 ['2_37_70', 0],
 ['2_37_70', 0],
 ['2_37_70', 0],
 ['2_37_70', 0],
 ['2_37_48_77', 0],
 ['2_48_79', 0],
 ['2_37_48', 1],
 ['2_48_79', 0],
 ['2_37_48', 1],
 ['2_37_48_77', 0],
 ['13', 0],
 ['13', 0],
 ['13', 0],
 ['13', 0],
 ['15', 1],
 ['15', 1],
 ['15', 1],
 [nan, 0]]